# Basic stats on the evolution of COVID-19 hospitalizations in departments

In [1]:
%load_ext lab_black
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
from datetime import timedelta, date
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
pd.plotting.register_matplotlib_converters()

In [3]:
covid_url = (
    "https://www.data.gouv.fr/fr/datasets/r/63352e38-d353-4b54-bfd1-f1b3ee1cabd7"
)
filter_dep = ["971", "972", "973", "974", "976"]  # only metropolitan France
figsize = (15, 10)

#### <br> Load table with COVID-19 data (figures by dates)

In [4]:
df = pd.read_csv(covid_url, sep=";", parse_dates=True, index_col=2)
df = df.query("sexe == 0")  # sum male/female
df.drop(columns=["sexe"], inplace=True)
df.query("dep not in @filter_dep", inplace=True)
df.sort_index(inplace=True)
df.head(2)

,dep,hosp,rea,rad,dc
jour,,,,,
2020-03-18,01,2,0,1,0
2020-03-18,02,41,10,18,11


In [5]:
last_day = df.index.max()
previous_day = last_day - timedelta(days=1)
print("Evolution from ", previous_day, " to ", last_day)

Evolution from  2020-03-29 00:00:00  to  2020-03-30 00:00:00


#### <br> Build a new dataframe with data from two latest days and the associated evolution

In [6]:
sdf = df.loc[last_day]
sdf.reset_index(inplace=True)
pdf = df.loc[previous_day]
pdf.reset_index(inplace=True)
frame = {"dep": sdf["dep"], "prev_hosp": pdf["hosp"], "last_hosp": sdf["hosp"]}
stats = pd.DataFrame(frame)
stats["diff"] = stats["last_hosp"] - stats["prev_hosp"]
stats["percent"] = stats["diff"] / stats["prev_hosp"] * 100
stats.head()

,dep,prev_hosp,last_hosp,diff,percent
0,01,55,67,12,21.818182
1,02,136,135,-1,-0.735294
2,03,23,30,7,30.434783
3,04,12,19,7,58.333333
4,05,39,38,-1,-2.564103


#### <br> Departments with highest number of hospitalizations on the latest day

In [7]:
stats.sort_values(by="last_hosp", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
75,75,2115,2217,102,4.822695
92,92,1169,1286,117,10.008554
94,94,961,1030,69,7.180021
68,68,999,1027,28,2.802803
69,69,907,985,78,8.599779


#### <br> Min, max, mean of evolution between the two latest days (in sum and %)

In [8]:
print(
    "Min: ",
    stats["diff"].min(),
    " Mean: ",
    stats["diff"].mean(),
    " Max: ",
    stats["diff"].max(),
)

Min:  -7  Mean:  17.09375  Max:  117


In [9]:
print(
    "Min %: ",
    stats.percent.min(),
    " Mean %: ",
    stats.percent.mean(),
    " Max %: ",
    stats.percent.max(),
)

Min %:  -11.11111111111111  Mean %:  13.047278849053937  Max %:  77.77777777777779


#### <br> List of departments with a drecrease of hospitalizations (last two days)

In [10]:
stats.query("diff < 0")

,dep,prev_hosp,last_hosp,diff,percent
1,02,136,135,-1,-0.735294
4,05,39,38,-1,-2.564103
19,21,218,211,-7,-3.211009
32,32,20,19,-1,-5.000000
48,48,9,8,-1,-11.111111
66,66,126,123,-3,-2.380952
70,70,66,63,-3,-4.545455


#### <br> Departments with highest evolution of hospitalizations (total number)

In [11]:
stats.sort_values(by="diff", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
92,92,1169,1286,117,10.008554
12,13,705,811,106,15.035461
93,93,758,863,105,13.852243
75,75,2115,2217,102,4.822695
69,69,907,985,78,8.599779


#### <br> Departments with highest rate of hospitalizations (%)

In [12]:
stats.sort_values(by="percent", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
17,18,18,32,14,77.777778
82,82,4,7,3,75.000000
41,41,29,48,19,65.517241
3,04,12,19,7,58.333333
37,37,58,81,23,39.655172


#### <br> Focus on some departments

In [13]:
stats.query("dep=='75'")

,dep,prev_hosp,last_hosp,diff,percent
75,75,2115,2217,102,4.822695


In [14]:
stats.query("dep=='71'")

,dep,prev_hosp,last_hosp,diff,percent
71,71,184,187,3,1.630435


In [15]:
stats.query("dep=='31'")

,dep,prev_hosp,last_hosp,diff,percent
31,31,185,214,29,15.675676
